# Monet using GAN

In [ ]:
import os
import matplotlib.pyplot as plt
import time 
import seaborn as sns
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# import tensorflow_addons as tfa
# from kaggle_datasets import KaggleDatasets

if os.path.exists(r'C:\Users\kuusnin\tempwork\temp\gan-getting-started'):
    datapath = r'C:\Users\kuusnin\tempwork\temp\gan-getting-started'
elif os.path.exists(r'C:\Users\nikok\Documents\Monet using GAN'):
    datapath = r'C:\Users\nikok\Documents\Monet using GAN'
else:
    datapath = r'/kaggle/input/gan-getting-started'
print(datapath)

In [ ]:
tf.__version__
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
except Exception as e:
    print("can't initialize tpu, using default, exception: " + str(e))
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)

In [ ]:
print("Available GPUs:", tf.config.list_physical_devices('GPU'))
strategy = tf.distribute.MirroredStrategy()

## Brief description of the problem and data (5 pts)

*Briefly describe the challenge problem and NLP. Describe the size, dimension, structure, etc., of the data.*

In [ ]:
monet_files = os.listdir(os.path.join(datapath, 'monet_tfrec'))
photo_files = os.listdir(os.path.join(datapath, 'photo_tfrec'))
monet_filenames = [os.path.join(datapath, 'monet_tfrec', f) for f in monet_files]
photo_filenames = [os.path.join(datapath, 'photo_tfrec', f) for f in photo_files]
print(20*'*', 'Monet paintings', 20*'*')
print('First filename:', monet_files[0], '\nNumber of files:', len(monet_files))
print(20*'*', 'Photos', 20*'*')
print('First filename:', photo_files[0], '\nNumber of files:', len(photo_files))

### Figure out the contents of the tfrec files

From the code below we can see that each record/example contains three fields: 
* target: label of the image. Not needed in this work 
* image_name: name of the image
* image: the actual image data

Both of the data sets appear to have the same structure. 

In [ ]:
# The following code is adapted from an answer from Microsoft Copilot
import tensorflow as tf
from google.protobuf.json_format import MessageToJson
import json

def iterate_record(dataset):
    # Initialize a counter
    record_count = 0
    # Iterate through the dataset and count the records
    for _ in dataset:
        record_count += 1
    print(f'Total number of records: {record_count}')
    
    # Iterate through the dataset and parse each record
    for raw_record in dataset.take(1):  # Adjust the number to read more records
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())
        json_message = MessageToJson(example)
        parsed_record = json.loads(json_message)
        print(json.dumps(parsed_record, indent=2)[0:500])

# Create a TFRecordDataset
print(20*'*', 'Monet paintings', 20*'*')
raw_monet_dataset = tf.data.TFRecordDataset(monet_filenames)
iterate_record(raw_monet_dataset)

print('\n' + 20*'*', 'Photos', 20*'*')
raw_photo_dataset = tf.data.TFRecordDataset(photo_filenames)
iterate_record(raw_photo_dataset)


We can parse the data now since the structure of the tfrecord is known.

In [ ]:
def normalize(image):
    return (tf.cast(image, tf.float32) / 127.5) - 1

def parse_tfrecord_fn(example):
    feature_description = {
        "image_name": tf.io.FixedLenFeature([], tf.string),
        "image": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, feature_description)
    example["image"] = tf.io.decode_jpeg(example["image"], channels=3)
    example["image"] = normalize(example["image"])
    return example

def decode_image(tf_image):
    return ((tf_image['image'].numpy() + 1) * 127.5).astype(int)

monet_dataset = raw_monet_dataset.map(parse_tfrecord_fn, num_parallel_calls=tf.data.AUTOTUNE).batch(32)
photo_dataset = raw_photo_dataset.map(parse_tfrecord_fn, num_parallel_calls=tf.data.AUTOTUNE).batch(32)

In [ ]:
sample_monet = next(iter(monet_dataset))
sample_photo = next(iter(photo_dataset))

In [ ]:
def get_dims(dataset, name):
    dims = []
    for data in dataset:
        dims.append(data['image'].numpy().shape)
    dims = np.array(dims)
    print(f'Number of images in {name} dataset:', np.sum(dims[:,0]))
    print(f'Unique shapes of the {name} data:', np.unique(dims[:,1:], axis=0))

get_dims(monet_dataset, 'Monet')
get_dims(photo_dataset, 'photos')

## Exploratory Data Analysis (EDA) — Inspect, Visualize and Clean the Data (15 pts)

*Show a few visualizations like histograms. Describe any data cleaning procedures. Based on your EDA, what is your plan of analysis?*

In [ ]:
fig = plt.figure(figsize=(12, 6))
i = 0
for i in range(4):
    data = sample_monet['image']
    ax = plt.subplot(2, 4, i + 1)
    plt.imshow(np.array((data[i]+1)*127.5).astype(int))
    i += 1
for data in range(4):
    data = sample_photo['image']
    ax = plt.subplot(2, 4, i + 1)
    plt.imshow(np.array((data[i]+1)*127.5).astype(int))
    i += 1
fig.suptitle('Examples of Monet and photos')
plt.show()

Let's check distribution of real image vs Monet paintings.

In [ ]:
def plot_histogram(dataset, title):
    rgb = []
    rgb = [decode_image(d) for d in dataset.take(9)]
    # for data in dataset:
    #     rgb.append(decode_image(data))
    rgb = np.array(rgb)
    print(rgb.shape)
    pic_channels = ['red', 'green', 'blue']
    rgb_df = pd.DataFrame()
    for i, clr in enumerate(pic_channels):
        df = pd.DataFrame(rgb[:,:,:,:,i].ravel(), columns=['val'])
        df['color'] = clr
        rgb_df = pd.concat([rgb_df, df])
    print('Minimum and maximmum values in ' + title + ' data: ', min(rgb_df.val), '&', max(rgb_df.val))
    sns.histplot(rgb_df.sample(1000), x='val', hue='color', bins=30, multiple='dodge', color=['green', 'blue', 'red'])
    plt.grid()
    plt.title('Histogram of ' + title)
    plt.show()


In [ ]:
plot_histogram(photo_dataset, title='sample photos')
plot_histogram(monet_dataset, title='Monet paintings')


After the EDA is done, we set the batch size of the dataset for training.

Below mostly from https://www.tensorflow.org/datasets/performances

In [ ]:
BATCH_SIZE = 32
# # ds = ds.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
# monet_dataset = monet_dataset.cache()
# # For true randomness, we set the shuffle buffer to the full dataset size.
# monet_dataset = monet_dataset.shuffle(number_of_monets*100).repeat(100)
# # Batch after shuffling to get unique batches at each epoch.
# monet_dataset = monet_dataset.batch(BATCH_SIZE, drop_remainder=True)
# monet_dataset = monet_dataset.prefetch(tf.data.experimental.AUTOTUNE)

# .shuffle(10000, reshuffle_each_iteration=True).repeat(repeats))

## Model Architecture (25 pts)

*Describe your model architecture and reasoning for why you believe that specific architecture would be suitable for this problem.*

### Discriminator
Source: https://keras.io/examples/generative/dcgan_overriding_train_step/

In [ ]:
my_discriminator = keras.Sequential(
    [
        keras.Input(shape=(256, 256, 3)),
        layers.Conv2D(int(64), kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(int(128), kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(int(128), kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Flatten(),
        layers.Dropout(0.2),
        layers.Dense(1, activation="sigmoid"),
    ],
    name="discriminator",
)
my_discriminator.summary()

In [ ]:
with strategy.scope():
    my_discriminator2 = keras.Sequential(
        [
            keras.Input(shape=(256, 256, 3)),
            layers.Conv2D(int(32), kernel_size=3, strides=2, padding="same"),
            layers.BatchNormalization(),
            layers.LeakyReLU(alpha=0.2),
            layers.Conv2D(int(64), kernel_size=3, strides=2, padding="same"),
            layers.BatchNormalization(),
            layers.LeakyReLU(alpha=0.2),
            layers.Conv2D(int(128), kernel_size=3, strides=2, padding="same"),
            layers.BatchNormalization(),
            layers.LeakyReLU(alpha=0.2),
            layers.Conv2D(int(256), kernel_size=3, strides=2, padding="same"),
            layers.BatchNormalization(),
            layers.LeakyReLU(alpha=0.2),
            layers.Conv2D(int(512), kernel_size=3, strides=2, padding="same"),
            layers.BatchNormalization(),
            layers.LeakyReLU(alpha=0.2),
            layers.Flatten(),
            layers.Dropout(0.2),
            layers.Dense(1, activation="sigmoid"),
        ],
        name="discriminator",
    )
    my_discriminator2.summary()

#### Generator

Source: https://keras.io/examples/generative/dcgan_overriding_train_step/

In [ ]:
with strategy.scope():
    latent_dim = 100
    my_generator = keras.Sequential(
        [
            keras.Input(shape=(latent_dim,)),
            layers.Dense(8 * 8 * 128),
            layers.Reshape((8, 8, 128)),
            layers.Conv2DTranspose(int(128), kernel_size=4, strides=2, padding="same"),
            layers.LeakyReLU(alpha=0.2),
            layers.Conv2DTranspose(int(256), kernel_size=4, strides=4, padding="same"),
            layers.LeakyReLU(alpha=0.2),
            layers.Conv2DTranspose(int(512), kernel_size=4, strides=4, padding="same"),
            layers.LeakyReLU(alpha=0.2),
            layers.Conv2D(3, kernel_size=5, padding="same", activation="sigmoid"),
        ],
        name="generator",
    )
    my_generator.summary()

In [ ]:
with strategy.scope():
    latent_dim = 100
    my_generator2 = keras.Sequential(
        [
            keras.Input(shape=(latent_dim,)),
            layers.Dense(16 * 16 * 512),
            layers.Reshape((16, 16, 512)),
            layers.Conv2DTranspose(int(256), kernel_size=3, strides=2, padding="same"),
            layers.BatchNormalization(),
            layers.LeakyReLU(alpha=0.2),
            layers.Conv2DTranspose(int(128), kernel_size=3, strides=2, padding="same"),
            layers.BatchNormalization(),
            layers.LeakyReLU(alpha=0.2),
            layers.Conv2DTranspose(int(64), kernel_size=3, strides=2, padding="same"),
            layers.BatchNormalization(),
            layers.LeakyReLU(alpha=0.2),
            layers.Conv2DTranspose(int(32), kernel_size=3, strides=2, padding="same"),
            layers.BatchNormalization(),
            layers.LeakyReLU(alpha=0.2),
            layers.Conv2D(3, kernel_size=5, padding="same", activation="tanh"),
        ],
        name="generator",
    )
    my_generator2.summary()

#### Training

Sources: 
https://www.kaggle.com/code/thuylinh225/generate-monet-images-using-dcgan
https://keras.io/examples/generative/dcgan_overriding_train_step/

In [ ]:
# create loss function for the generator
with strategy.scope():
    def generator_loss(fake_output):
        # cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
        cross_entropy = tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
        return cross_entropy(tf.ones_like(fake_output), fake_output)

    # create loss function for the discriminator
    def discriminator_loss(real_output, fake_output):
        # cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
        cross_entropy = tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
        real_loss = cross_entropy(tf.ones_like(real_output), real_output)
        fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
        total_loss = real_loss + fake_loss
        return total_loss

#### Callbacks

Source: https://keras.io/examples/generative/dcgan_overriding_train_step/

In [ ]:
# Create two separate optimizers for the generator and discriminator
with strategy.scope():
    generator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
    discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)

In [ ]:
# Set the hyperparameters to be used for training
EPOCHS = 201
BATCH_SIZE = 32
noise_dim = 100
shape_dim = [256,256,3]

In [ ]:
class DCGAN_model:
    def __init__(self, noise_dim, EPOCHS, BATCH_SIZE, generator, discriminator, dataset):  
        self.noise_dim = noise_dim
        self.EPOCHS = EPOCHS
        self.BATCH_SIZE = BATCH_SIZE
        self.generator = generator
        self.discriminator = discriminator
        self.dataset = dataset
    
    @tf.function
    def train(self, images):
    
    # Create random noise vector
        noise = tf.random.normal([images.shape[0], noise_dim])

        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        
        # generate images use random noise vector
            generated_images = self.generator(noise, training=True)

            # use discriminator to evaluate the real and fake images
            real_output = self.discriminator(images, training=True)
            fake_output = self.discriminator(generated_images, training=True)

            # compute generator loss and discriminator loss
            gen_loss = generator_loss(fake_output)
            disc_loss = discriminator_loss(real_output, fake_output)

            # Compute gradients
            gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
            gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

            # Update optimizers
            generator_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
            discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))
        
        return (gen_loss + disc_loss) * 0.5
    
    @tf.function
    def distributed_train(self, images):
        per_replica_losses = strategy.run(self.train, args=(images,))
        return strategy.reduce(tf.distribute.ReduceOp.MEAN, per_replica_losses, axis=None)
    
    def generate_images(self):
        noise = tf.random.normal([self.BATCH_SIZE, self.noise_dim]) 
        predictions = self.generator.predict(noise)
        return predictions
        
    def generate_and_plot_images(self):      
        image = self.generate_images()
        gen_imgs = 0.5 * image + 0.5
        fig = plt.figure(figsize=(10, 10))
        for i in range(25):
            plt.subplot(5, 5, i+1)
            plt.imshow(gen_imgs[i, :, :, :])
            plt.axis('off')
        plt.show()

    def train_loop(self):
        e_ls = []
        mean_ls = []
        for epoch in range(self.EPOCHS):
            start = time.time()
            print('Epoch:', epoch+1, end='\r')

            total_loss = 0.0
            num_batches = 0

            for image_batch in self.dataset:
                loss = self.distributed_train(image_batch['image'])
                total_loss += tf.reduce_mean(loss)
                num_batches += 1
            mean_loss = total_loss / num_batches

            if (epoch+1) % 200 == 0:                                  
                print ('Time for epoch {} is {} sec, mean loss is {}'.format(epoch + 1, time.time()-start, mean_loss))
                self.generate_and_plot_images()
                
                e_ls.append(epoch+1)
                mean_ls.append(mean_loss)
        print("\nMean Loss for every 200 epochs: \n")
        table = pd.DataFrame({"Epoch": e_ls, "Mean Loss": np.array(mean_ls)})
        return table

#### Training

Override the train step.

Source: https://keras.io/examples/generative/dcgan_overriding_train_step/

In [ ]:
# train, visualize and print out the result for DCGAN model
gan1 = DCGAN_model(noise_dim, EPOCHS, BATCH_SIZE, my_generator2, my_discriminator2, monet_dataset)
res1 = gan1.train_loop()
res1

## Results and Analysis (35 pts)

*Run hyperparameter tuning, try different architectures for comparison, apply techniques to improve training or performance, and discuss what helped.*

*Includes results with tables and figures. There is an analysis of why or why not something worked well, troubleshooting, and a hyperparameter optimization procedure summary.*

## Conclusion (15 pts)

*Discuss and interpret results as well as learnings and takeaways. What did and did not help improve the performance of your models? What improvements could you try in the future?*

# Sources

https://www.kaggle.com/code/amyjang/monet-cyclegan-tutorial
https://keras.io/examples/generative/dcgan_overriding_train_step
